In [1]:
!pip install flask flask-ngrok
!pip install easyocr imutils opencv-python


In [1]:
import cv2
from tkinter import Tk, Label, Button, filedialog, Canvas, Toplevel, Frame, messagebox
from tkinter.ttk import Progressbar
from PIL import Image, ImageTk
import easyocr
import numpy as np
import imutils

# Initialize Tkinter GUI
class NumberPlateApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Number Plate Detection")
        self.root.geometry("900x700")
        self.root.config(bg="#1e1e2f")

        # Canvas to show the image
        self.canvas = Canvas(root, width=800, height=400, bg="grey", highlightthickness=1, highlightbackground="black")
        self.canvas.pack(pady=20)

        # Buttons and Options
        button_frame = Frame(root, bg="#1e1e2f")
        button_frame.pack()

        self.button_open = Button(button_frame, text="Upload Image", command=self.upload_image, bg="#4CAF50", fg="white", font=("Arial", 12))
        self.button_open.grid(row=0, column=0, padx=10, pady=10)

        self.button_detect = Button(button_frame, text="Detect Plate", command=self.detect_plate, bg="#FF5722", fg="white", font=("Arial", 12))
        self.button_detect.grid(row=0, column=1, padx=10, pady=10)

        self.button_save = Button(button_frame, text="Save Plate", command=self.save_plate_image, bg="#9C27B0", fg="white", font=("Arial", 12))
        self.button_save.grid(row=0, column=2, padx=10, pady=10)

        self.button_clear = Button(button_frame, text="Clear All", command=self.clear_canvas, bg="#f44336", fg="white", font=("Arial", 12))
        self.button_clear.grid(row=0, column=3, padx=10, pady=10)

        # Result Label
        self.label_result = Label(root, text="Detected Text: ", bg="#1e1e2f", fg="white", font=("Arial", 14))
        self.label_result.pack(pady=10)

        # Status Bar
        self.status = Label(root, text="Welcome to Number Plate Detection", bd=1, relief="sunken", anchor="w", bg="#333333", fg="white", font=("Arial", 10))
        self.status.pack(side="bottom", fill="x")

        # Instance Variables
        self.image_path = None
        self.cropped_image = None

    def update_status(self, message):
        self.status.config(text=message)

    def upload_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
        if file_path:
            self.image_path = file_path
            img = Image.open(file_path)
            img.thumbnail((800, 400))
            self.img_tk = ImageTk.PhotoImage(img)
            self.canvas.create_image(400, 200, image=self.img_tk)
            self.label_result.config(text="Detected Text: ")
            self.update_status("Image uploaded successfully!")

    def detect_plate(self):
        if not self.image_path:
            self.update_status("Please upload or capture an image first!")
            return

        self.update_status("Processing the image...")
        img = cv2.imread(self.image_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        bfilter = cv2.bilateralFilter(gray, 11, 17, 17)  # Noise reduction
        edged = cv2.Canny(bfilter, 30, 200)

        # Find contours
        keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = imutils.grab_contours(keypoints)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

        location = None
        for contour in contours:
            approx = cv2.approxPolyDP(contour, 10, True)
            if len(approx) == 4:
                location = approx
                break

        if location is None:
            self.label_result.config(text="Number Plate Not Detected!")
            self.update_status("Detection failed!")
            return

        mask = np.zeros(gray.shape, np.uint8)
        new_image = cv2.drawContours(mask, [location], 0, 255, -1)
        new_image = cv2.bitwise_and(img, img, mask=mask)

        (x, y) = np.where(mask == 255)
        (x1, y1) = (np.min(x), np.min(y))
        (x2, y2) = (np.max(x), np.max(y))
        self.cropped_image = gray[x1:x2 + 1, y1:y2 + 1]

        # Use EasyOCR to detect text
        reader = easyocr.Reader(['en'])
        result = reader.readtext(self.cropped_image)
        if result:
            detected_text = result[0][-2]
            self.label_result.config(text=f"Detected Text: {detected_text}")
            self.update_status("Number plate detected!")
        else:
            self.label_result.config(text="No text detected!")
            self.update_status("Text detection failed!")

    def save_plate_image(self):
        if self.cropped_image is not None:
            save_path = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG files", "*.jpg"), ("PNG files", "*.png")])
            if save_path:
                cv2.imwrite(save_path, self.cropped_image)
                self.update_status("Cropped plate image saved successfully!")
        else:
            self.update_status("No cropped image to save!")

    def clear_canvas(self):
        self.canvas.delete("all")
        self.label_result.config(text="Detected Text: ")
        self.image_path = None
        self.cropped_image = None
        self.update_status("Canvas cleared!")

# Run the Tkinter App
if __name__ == "__main__":
    root = Tk()
    app = NumberPlateApp(root)
    root.mainloop()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
